# A Guided Tour of Ray Core: Parallel Iterators

[*Parallel Iterators*](https://docs.ray.io/en/latest/iter.html) provide an API for simple data ingest and processing.
This is fully serializable and can operate over infinite sequences of items.
Transformations are based on method chaining, an iterators gets passed to remote tasks and actors to represent *data shards*.

---

First, let's start Ray…

In [1]:
from icecream import ic
import logging
import ray

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

File descriptor limit 256 is too low for production servers and may result in connection errors. At least 8192 is recommended. --- Fix with 'ulimit -n 8192'


{'node_ip_address': '192.168.1.65',
 'raylet_ip_address': '192.168.1.65',
 'redis_address': '192.168.1.65:6379',
 'object_store_address': '/tmp/ray/session_2021-02-23_15-42-26_681054_93562/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-23_15-42-26_681054_93562/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-02-23_15-42-26_681054_93562',
 'metrics_export_port': 58232,
 'node_id': '22e4f838645a5ef3f881b31b0c9f6159a0d7fb83'}

## Parallel Iterators

Set up for this example...

In [2]:
import numpy as np

In [3]:
@ray.remote
def train (data_shard):
    for batch in data_shard:
        print("train on", batch)  # perform model update with batch

In [4]:
para_iter = (
    ray.util.iter.from_range(5, num_shards=2, repeat=True)
        .batch(3)
        .for_each(np.array)
)

In [5]:
work = [train.remote(shard) for shard in para_iter.shards()]

(pid=93582) train on [2 3 4]
(pid=93583) train on [0 1 0]
(pid=93583) train on [1 0 1]
(pid=93583) train on [0 1 0]
(pid=93583) train on [1 0 1]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93583) 
(pid=93583) train on [0 1 0]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(pid=93582) train on [2 3 4]
(

In [ ]:
ray.get(work)